In [3]:
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import pandas as pd
# import rpy2_arrow.pyarrow_rarrow as pyra
%load_ext rpy2.ipython
rpy2.robjects.numpy2ri.activate()

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
%%R
library(lsa)
library(vegan)
library(ape)
# library(psych)
library(ggplot2)
# library(dendextend)
# library(circlize)
# library(grid)
library(tidyr)

R[write to console]: Loading required package: SnowballC

R[write to console]: Loading required package: permute

R[write to console]: Loading required package: lattice

R[write to console]: This is vegan 2.6-4



In [5]:
def get_pairwise_df(df):
    concept1 = []
    concept2 = []
    scores = []
    for i, row in df.iterrows():
        left = row['prompt'].split('similar is ')[-1].split(' and ')[0]
        right = row['prompt'].split('similar is ')[-1].split(' and ')[1][:-1]
        score = 7-row['score']
        concept1.append(left)
        concept2.append(right)
        scores.append(score)
    pairwise_df = pd.DataFrame({'concept_1': concept1, 'concept_2': concept2, 'response': scores})
    pairwise_dsm = pairwise_df.groupby(['concept_1', 'concept_2']).mean().reset_index()
    pairwise_dsm = pd.pivot(pairwise_dsm, index = 'concept_1', columns = 'concept_2', values = 'response')
    return pairwise_dsm
    

## feature list humans (leuven) vs feature list gpt

### old code

In [6]:
%%R
# set working dir to the dir the script is in
setwd("/home/siddsuresh97/Projects/conceptual_representations_gpt/emnlp/")

# Load data
leuven_animals = read.csv('data/feature_listing/animal_leuven_norms.csv')
leuven_tools = read.csv('data/feature_listing/artifacts_leuven_norms.csv')
clip = read.csv('data/clip/clip_embeddings.csv')
w2v = read.csv('data/word2vec/word2vec_embeddings.csv')
human_triplet_mds = read.csv('data/triplets/summer_data/human_reptile_tool_embedding_df.csv')
gpt_triplet_mds = read.csv('data/triplets/summer_data/gpt_reptile_tool_embedding_df_temp0_cogsci2023.csv')
gpt_embed <- read.csv('data/gpt/gpt_embeddings_text-similarity-davinci-001.csv')

In [7]:
%%R 
# clip_data
clip = clip[order(clip$X), ]
clip_dsm =1-lsa::cosine(t(data.matrix(clip[,2:ncol(clip)])))
clip_dsm = clip_dsm[order(clip$X),order(clip$X)]
clip_mds =  cmdscale(clip_dsm,k = 3)
rownames(clip_dsm) <- clip$X
rownames(clip_mds) <- clip$X
clip_mds <- clip_mds[order(rownames(clip_mds)),]

In [8]:
%%R 
# gpt_embed_data
gpt_embed = gpt_embed[order(gpt_embed$X), ]
gpt_embed_dsm =1-lsa::cosine(t(data.matrix(gpt_embed[,2:ncol(gpt_embed)])))
gpt_embed_dsm = gpt_embed_dsm[order(gpt_embed$X),order(gpt_embed$X)]
gpt_embed_mds =  cmdscale(gpt_embed_dsm,k = 3)
rownames(gpt_embed_dsm) <- gpt_embed$X
rownames(gpt_embed_mds) <- gpt_embed$X
gpt_embed_dsm <- gpt_embed_dsm[order(rownames(gpt_embed_dsm)),]

In [9]:
 %%R 
#word2vec_data
w2v = w2v[order(w2v$X), ]
w2v_dsm =1-lsa::cosine(t(data.matrix(w2v[,2:ncol(w2v)])))
w2v_dsm = w2v_dsm[order(w2v$X),order(w2v$X)]
# <!-- w2v_dsm = dist(w2v[,2:ncol(w2v)) -->
w2v_dsm =  cmdscale(w2v_dsm,k = 3)
rownames(w2v_dsm) <- w2v$X
rownames(w2v_dsm) <- w2v$X
w2v_mds <- w2v_mds[order(rownames(w2v_mds)),]

In [10]:
%%R 
#leuven_data
leuven = merge(leuven_animals, leuven_tools, by = 'X', all = TRUE)
leuven[is.na(leuven)] = 0
leuven_cosine_dist = 1-  lsa::cosine(t(data.matrix(leuven[,2:ncol(leuven)])))
leuven_dsm = leuven_cosine_dist[order(leuven$X),order(leuven$X)]
leuven_mds <- cmdscale(leuven_cosine_dist,k = 3)
rownames(leuven_cosine_dist) <- leuven$X
rownames(leuven_mds) <- leuven$X
leuven_mds <- leuven_mds[order(rownames(leuven_mds)),]

In [11]:
%%R 
#flan_xl_verified
flan_xl_verified = read.csv('data/feature_listing/flan_xl_wide.csv')
flan_xl_dsm = 1-  lsa::cosine(t(data.matrix(flan_xl_verified[,2:ncol(flan_xl_verified)])))
flan_xl_dsm = flan_xl_dsm[order(flan_xl_verified$Concept),order(flan_xl_verified$Concept)]
flan_xl_mds <- cmdscale(flan_xl_dsm,k = 3)
rownames(flan_xl_dsm) <- flan_xl_verified$Concept
rownames(flan_xl_mds) = rownames(flan_xl_dsm)
flan_xl_mds <- flan_xl_mds[order(rownames(flan_xl_mds)),]

In [12]:
%%R 
#flan_xxl_verified
flan_xxl_verified = read.csv('data/feature_listing/flan_xxl_wide.csv')
flan_xxl_dsm = 1-  lsa::cosine(t(data.matrix(flan_xxl_verified[,2:ncol(flan_xxl_verified)])))
flan_xxl_dsm = flan_xxl_dsm[order(flan_xxl_verified$Concept),order(flan_xxl_verified$Concept)]
flan_xxl_mds <- cmdscale(flan_xxl_dsm,k = 3)
rownames(flan_xxl_dsm) <- flan_xxl_verified$Concept
rownames(flan_xxl_mds) = rownames(flan_xxl_dsm)
flan_xxl_mds <- flan_xxl_mds[order(rownames(flan_xxl_mds)),]

In [13]:
%%R 
#gpt unverified
gpt_summer_no_mc = read.csv('data/feature_listing/gpt_response_unverified_wide_no_matrix_completion.csv')
unverified_gpt_dsm = 1-  lsa::cosine(t(data.matrix(gpt_summer_no_mc[,2:ncol(gpt_summer_no_mc)])))
unverified_gpt_dsm = unverified_gpt_dsm[order(gpt_summer_no_mc$Concept),order(gpt_summer_no_mc$Concept)]
unverified_gpt_mds_summer_no_mc <- cmdscale(unverified_gpt_dsm,k = 3)
rownames(unverified_gpt_dsm) <- gpt_summer_no_mc$Concept
rownames(unverified_gpt_mds_summer_no_mc) = rownames(unverified_gpt_dsm)
unverified_gpt_mds_summer_no_mc <- unverified_gpt_mds_summer_no_mc[order(rownames(unverified_gpt_mds_summer_no_mc)),]

In [14]:
%%R 
# gpt verified 
gpt_summer_no_mc = read.csv('data/feature_listing/gpt_response_yes_wide_no_matrix_completion.csv')
verified_gpt_dsm = 1-  lsa::cosine(t(data.matrix(gpt_summer_no_mc[,2:ncol(gpt_summer_no_mc)])))
verified_gpt_dsm = verified_gpt_dsm[order(gpt_summer_no_mc$Concept),order(gpt_summer_no_mc$Concept)]
verified_gpt_mds_summer_no_mc <- cmdscale(verified_gpt_dsm,k = 3)
rownames(verified_gpt_dsm) <- gpt_summer_no_mc$Concept
rownames(verified_gpt_mds_summer_no_mc) = rownames(verified_gpt_dsm)
verified_gpt_mds_summer_no_mc <- verified_gpt_mds_summer_no_mc[order(rownames(verified_gpt_mds_summer_no_mc)),]

In [15]:
%%R 
# human triplets
rownames(human_triplet_mds) <- human_triplet_mds$concept
human_triplet_mds <- human_triplet_mds[order(rownames(human_triplet_mds)),]
human_triplet_mds <- human_triplet_mds[, 3:5]
human_triplet_cosine_dist = 1-  lsa::cosine(t(data.matrix(human_triplet_mds)))
human_triplet_dsm = human_triplet_cosine_dist
rownames(human_triplet_cosine_dist) <- human_triplet_mds$concept

In [16]:
%%R 
# gpt triplets
rownames(gpt_triplet_mds) <- gpt_triplet_mds$concept
gpt_triplet_mds <- gpt_triplet_mds[order(rownames(gpt_triplet_mds)),]
gpt_triplet_mds <- gpt_triplet_mds[, 3:5]
gpt_triplet_cosine_dist = 1-  lsa::cosine(t(data.matrix(gpt_triplet_mds)))
gpt_triplet_dsm = gpt_triplet_cosine_dist
rownames(gpt_triplet_cosine_dist) <- gpt_triplet_mds$concept

### new code

In [17]:
%%R
gpt_003_pairwise_dsm = read.csv('data/pairwise/text-davinci-003/response_davinci-003_0.7_mean_matrix.csv')
gpt_003_pairwise_dsm = gpt_003_pairwise_dsm[1:(nrow(gpt_003_pairwise_dsm)), 2:(ncol(gpt_003_pairwise_dsm)), drop = FALSE]
colnames(gpt_003_pairwise_dsm) <- NULL
gpt_003_pairwise_dsm = (t(as.matrix(gpt_003_pairwise_dsm)) + as.matrix(gpt_003_pairwise_dsm))/2
gpt_003_pairwise_dsm = 7 - gpt_003_pairwise_dsm
gpt_003_pairwise_mds = cmdscale(as.matrix(gpt_003_pairwise_dsm),k = 3)


gpt_002_pairwise_dsm = read.csv('data/pairwise/text-davinci-002/mean_response_davinci-002.csv')
gpt_002_pairwise_dsm = gpt_002_pairwise_dsm[1:(nrow(gpt_002_pairwise_dsm)), 2:(ncol(gpt_002_pairwise_dsm)), drop = FALSE]
colnames(gpt_002_pairwise_dsm) <- NULL
gpt_002_pairwise_dsm = (t(as.matrix(gpt_002_pairwise_dsm)) + as.matrix(gpt_002_pairwise_dsm))/2
gpt_002_pairwise_dsm = 7 - gpt_002_pairwise_dsm
gpt_002_pairwise_mds = cmdscale(as.matrix(gpt_002_pairwise_dsm),k = 3)

In [18]:
human_pairwise_dsm = pd.read_csv('data/pairwise/human/human_rating_df.csv')
# keep columns ['concept_1', 'concept_2', 'response_clean']
human_pairwise_dsm = human_pairwise_dsm[['concept_1', 'concept_2', 'response_clean']]

# group by concept_1 and concept_2 and take mean of response_clean
human_pairwise_dsm = human_pairwise_dsm.groupby(['concept_1', 'concept_2']).mean().reset_index()
human_pairwise_dsm = pd.pivot(human_pairwise_dsm, index = 'concept_1', columns = 'concept_2', values = 'response_clean')
# wherever there is a nan, replace with 7
human_pairwise_dsm = human_pairwise_dsm.fillna(6)

#remove row and column names
human_pairwise_dsm.index.name = None
human_pairwise_dsm.columns.name = None

# convert from python to R
def numpy_mat_to_R(var_name, var):
    nr,nc = var.shape
    Br = ro.r.matrix(var, nrow=nr, ncol=nc)
    ro.r.assign(var_name, Br)
numpy_mat_to_R('human_pairwise_dsm', human_pairwise_dsm.values)



In [19]:
human_pairwise_dsm.to_csv('data/pairwise/human/mean_human_pairwise_similarity.csv')

In [20]:
%%R 
human_pairwise_dsm = (t(as.matrix(human_pairwise_dsm)) + as.matrix(human_pairwise_dsm))/2
human_pairwise_dsm = 6 - human_pairwise_dsm
human_pairwise_mds = cmdscale(human_pairwise_dsm,k = 3)

In [21]:
%%R
# flan-xxl
flan_xxl_triplet_mds <- read.csv('embeddings_data/flan-t5-xxl.csv')
flan_xxl_cot_triplet_mds <- read.csv('embeddings_data/flan_xxl_cot.csv')


#flan-ul2
flan_ul2_triplet_mds <- read.csv('embeddings_data/flan-ul2.csv')
flan_ul2_cot_triplet_mds <- read.csv('embeddings_data/flan-ul2-cot.csv')



# gpt
gpt_003_cot_triplet_mds <- read.csv('embeddings_data/text-davinci-003_cot.csv')
gpt_002_cot_triplet_mds <- read.csv('embeddings_data/text-davinci-002_cot.csv')



In [22]:
import re
def get_pairwise_df_gpt(df):
    concept1 = []
    concept2 = []
    scores = []
    for i, row in df.iterrows():
        left = row['prompt'].split('similar is ')[-1].split(' and ')[0]
        right = row['prompt'].split('similar is ')[-1].split(' and ')[1].split('?')[0]
        # if type(row['response'].split('The answer is ')[-1][0]) == int:
        try:
            row['score'] = int(row['response'].split('answer is ')[-1][0])
        except:
            row['score'] = row['response'].split('answer is ')[-1]
            if 'very dissimilar' in row['score'].lower():
                row['score'] = 2
            elif 'extremely dissimilar' in row['score'].lower():
                row['score'] = 1
            elif 'similar' in row['score'].lower():
                row['score'] = 6
            elif 'dissimilar' in row['score'].lower():
                row['score'] = 2
            elif 'neutral' in row['score'].lower():
                row['score'] = 5
        # try:
        #     score = 7-int(row['score'])
        # except:
        #     print(row['prompt'])
        try:
            score = 7 - int(row['score'])
        except:
            matches = re.findall(r'\d+\.?\d*', row['score'])
            if len(matches) == 0:
                score = 4
            elif '.' in matches[0]:
                matches[0] = matches[0].split('.')[0]
            elif len(matches[0]) > 1:
                score = 4
            else:
                score = int(matches[0])
            score = 7 - score
        concept1.append(left)
        concept2.append(right)
        scores.append(score)
    pairwise_df = pd.DataFrame({'concept_1': concept1, 'concept_2': concept2, 'response': scores})
    pairwise_dsm = pairwise_df.groupby(['concept_1', 'concept_2']).mean().reset_index()
    pairwise_dsm.loc[pairwise_dsm['concept_1'] == pairwise_dsm['concept_2'], 'response'] = 0
    pairwise_dsm = pd.pivot(pairwise_dsm, index = 'concept_1', columns = 'concept_2', values = 'response')
    return pairwise_dsm

In [23]:
%%R
rownames(flan_xxl_triplet_mds) <- flan_xxl_triplet_mds$concept
flan_xxl_triplet_mds <- flan_xxl_triplet_mds[order(rownames(flan_xxl_triplet_mds)),]
flan_xxl_triplet_mds <- flan_xxl_triplet_mds[, 3:5]
flan_xxl_triplet_dsm = 1-  lsa::cosine(t(data.matrix(flan_xxl_triplet_mds)))


rownames(flan_xxl_cot_triplet_mds) <- flan_xxl_cot_triplet_mds$concept
flan_xxl_cot_triplet_mds <- flan_xxl_cot_triplet_mds[order(rownames(flan_xxl_cot_triplet_mds)),]
flan_xxl_cot_triplet_mds <- flan_xxl_cot_triplet_mds[, 3:5]
flan_xxl_cot_triplet_dsm = 1-  lsa::cosine(t(data.matrix(flan_xxl_cot_triplet_mds)))

rownames(flan_ul2_triplet_mds) <- flan_ul2_triplet_mds$concept
flan_ul2_triplet_mds <- flan_ul2_triplet_mds[order(rownames(flan_ul2_triplet_mds)),]
flan_ul2_triplet_mds <- flan_ul2_triplet_mds[, 3:5]
flan_ul2_triplet_dsm = 1-  lsa::cosine(t(data.matrix(flan_ul2_triplet_mds)))

rownames(flan_ul2_cot_triplet_mds) <- flan_ul2_cot_triplet_mds$concept
flan_ul2_cot_triplet_mds <- flan_ul2_cot_triplet_mds[order(rownames(flan_ul2_cot_triplet_mds)),]
flan_ul2_cot_triplet_mds <- flan_ul2_cot_triplet_mds[, 3:5]
flan_ul2_cot_triplet_dsm = 1-  lsa::cosine(t(data.matrix(flan_ul2_cot_triplet_mds)))

rownames(gpt_003_cot_triplet_mds) <- gpt_003_cot_triplet_mds$concept
gpt_003_cot_triplet_mds <- gpt_003_cot_triplet_mds[order(rownames(gpt_003_cot_triplet_mds)),]
gpt_003_cot_triplet_mds <- gpt_003_cot_triplet_mds[, 3:5]
gpt_003_cot_triplet_dsm = 1-  lsa::cosine(t(data.matrix(gpt_003_cot_triplet_mds)))


rownames(gpt_002_cot_triplet_mds) <- gpt_002_cot_triplet_mds$concept
gpt_002_cot_triplet_mds <- gpt_002_cot_triplet_mds[order(rownames(gpt_002_cot_triplet_mds)),]
gpt_002_cot_triplet_mds <- gpt_002_cot_triplet_mds[, 3:5]
gpt_002_cot_triplet_dsm = 1-  lsa::cosine(t(data.matrix(gpt_002_cot_triplet_mds)))


In [24]:
#flan-xxl
flan_xxl_pairwise = pd.read_csv('data/pairwise/flan-xxl/flan-t5-xxl_pairwise.csv', header=None)
flan_xxl_pairwise.columns = ['prompt', 'score']
flan_xxl_pairwise_dsm = get_pairwise_df(flan_xxl_pairwise)
def numpy_mat_to_R(var_name, var):
    nr,nc = var.shape
    Br = ro.r.matrix(var, nrow=nr, ncol=nc)
    ro.r.assign(var_name, Br)
numpy_mat_to_R('flan_xxl_pairwise_dsm', flan_xxl_pairwise_dsm.values)


#flan-ul2
flan_ul2_pairwise = pd.read_csv('data/pairwise/flan-ul2/flan-ul2_pairwise.csv', header=None)
flan_ul2_pairwise.columns = ['prompt', 'score']
flan_ul2_pairwise_dsm = get_pairwise_df(flan_ul2_pairwise)
def numpy_mat_to_R(var_name, var):
    nr,nc = var.shape
    Br = ro.r.matrix(var, nrow=nr, ncol=nc)
    ro.r.assign(var_name, Br)
numpy_mat_to_R('flan_ul2_pairwise_dsm', flan_ul2_pairwise_dsm.values)

flan_ul2_pairwise_cot = pd.read_csv('data/pairwise/flan-ul2/flan-ul2_cot_pairwise.csv', header=None)
flan_ul2_pairwise_cot.columns = ['prompt', 'response', 'score']
flan_ul2_pairwise_cot_dsm = get_pairwise_df_gpt(flan_ul2_pairwise_cot)
numpy_mat_to_R('flan_ul2_pairwise_cot_dsm', flan_ul2_pairwise_cot_dsm.values)


#text-davinci-003
gpt_003_pairwise_cot = pd.read_csv('data/pairwise/text-davinci-003/text-davinci-003_cot_pairwise.csv', header=None)
gpt_003_pairwise_cot.columns = ['prompt', 'response', 'score']
gpt_003_pairwise_cot_dsm = get_pairwise_df_gpt(gpt_003_pairwise_cot)
numpy_mat_to_R('gpt_003_pairwise_cot_dsm', gpt_003_pairwise_cot_dsm.values)


#text-davinci-002
gpt_002_pairwise_cot =  pd.read_csv('data/pairwise/text-davinci-002/gpt-semantics_cot_pairwise.csv')
gpt_002_pairwise_cot.columns = ['prompt', 'response', 'score']
gpt_002_pairwise_cot_dsm = get_pairwise_df_gpt(gpt_002_pairwise_cot)
gpt_002_pairwise_cot_dsm.fillna(0, inplace=True)
numpy_mat_to_R('gpt_002_pairwise_cot_dsm', gpt_002_pairwise_cot_dsm.values)




In [25]:
%%R
flan_xxl_pairwise_dsm = (t(as.matrix(flan_xxl_pairwise_dsm)) + as.matrix(flan_xxl_pairwise_dsm))/2
flan_xxl_pairwise_mds = cmdscale(flan_xxl_pairwise_dsm,k = 3)

flan_ul2_pairwise_dsm = (t(as.matrix(flan_ul2_pairwise_dsm)) + as.matrix(flan_ul2_pairwise_dsm))/2
flan_ul2_pairwise_mds = cmdscale(flan_ul2_pairwise_dsm,k = 3)

flan_ul2_pairwise_cot_dsm = (t(as.matrix(flan_ul2_pairwise_cot_dsm)) + as.matrix(flan_ul2_pairwise_cot_dsm))/2
flan_ul2_pairwise_cot_mds = cmdscale(flan_ul2_pairwise_cot_dsm,k = 3)


gpt_003_pairwise_cot_dsm = (t(as.matrix(gpt_003_pairwise_cot_dsm)) + as.matrix(gpt_003_pairwise_cot_dsm))/2
gpt_003_pairwise_cot_mds = cmdscale(gpt_003_pairwise_cot_dsm,k = 3)

gpt_002_pairwise_cot_dsm = (t(as.matrix(gpt_002_pairwise_cot_dsm)) + as.matrix(gpt_002_pairwise_cot_dsm))/2
gpt_002_pairwise_cot_mds = cmdscale(gpt_002_pairwise_cot_dsm,k = 3)

## pairwise protest all elements

In [41]:
%%R 

matrices = list(leuven_mds, w2v_mds, clip_mds, unverified_gpt_mds_summer_no_mc, verified_gpt_mds_summer_no_mc, human_triplet_mds, gpt_triplet_mds, gpt_embed_mds, flan_xl_mds, flan_xxl_mds, human_pairwise_mds, gpt_003_pairwise_mds, flan_xxl_triplet_mds, flan_xxl_cot_triplet_mds, flan_ul2_triplet_mds, flan_ul2_cot_triplet_mds, gpt_003_cot_triplet_mds, gpt_002_cot_triplet_mds, flan_ul2_pairwise_mds, flan_ul2_pairwise_cot_mds, gpt_003_pairwise_cot_mds, gpt_002_pairwise_cot_mds)

## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("leuven", "w2v", "clip", "unverified_gpt_summer_no_mc", "verified_gpt_summer_no_mc", "human_triplet", "gpt_triplet", "gpt_embed_mds", "flan_xl_mds", "flan_xxl_mds", "human_pairwise_mds", "gpt_003_pairwise_mds", "flan_xxl_triplet_mds", "flan_xxl_cot_triplet_mds", "flan_ul2_triplet_mds", "flan_ul2_cot_triplet_mds", "gpt_003_cot_triplet_mds", "gpt_002_cot_triplet_mds", "flan_ul2_pairwise_mds", "flan_ul2_cot_pairwise_mds", "gpt_003_pairwise_cot_mds", "gpt_002_pairwise_cot_mds")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        print(k)
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/mds_the_whole_correlation_table.csv", sep=",")

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 

## pairwise protest all elements - cogsci

In [74]:
%%R 

matrices = list(leuven_mds, w2v_mds, clip_mds, unverified_gpt_mds_summer_no_mc, verified_gpt_mds_summer_no_mc, human_triplet_mds, gpt_triplet_mds, gpt_embed_mds, flan_xl_mds, flan_xxl_mds, human_pairwise_mds, gpt_003_pairwise_mds)
## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("leuven", "w2v", "clip", "unverified_gpt_summer_no_mc", "verified_gpt_summer_no_mc", "human_triplet", "gpt_triplet", "gpt_embed_mds", "flan_xl_mds", "flan_xxl_mds", "human_pairwise_mds", "gpt_003_pairwise_mds")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/correlation_table.csv", sep=",")

R[write to console]: Error in withVisible({ : object 'unverified_gpt_mds' not found




Error in withVisible({ : object 'unverified_gpt_mds' not found


RInterpreterError: Failed to parse and evaluate line '\nmatrices = list(leuven_mds, w2v_mds, clip_mds, unverified_gpt_mds, verified_gpt_mds, human_triplet_mds, gpt_triplet_mds, gpt_embed_mds, flan_xl_mds, flan_xxl_mds, human_pairwise_mds, gpt_003_pairwise_mds)\n## Create an empty list to store the results\nresults = list()\n\n## Iterate through all combinations of the matrices\nfor (i in 1:(length(matrices))) {\n    for (j in 1:(length(matrices))) {\n        # Pass the two matrices to the protest function and store the result\n        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)\n        results = c(results, sqrt(1-result$ss))\n    }\n}\n\n## make the results into a table\n## Create a data frame with all possible combinations of matrix names\nmatrix_names = list("leuven", "w2v", "clip", "unverified_gpt_summer_no_mc", "verified_gpt_summer_no_mc", "human_triplet", "gpt_triplet", "gpt_embed_mds", "flan_xl_mds", "flan_xxl_mds", "human_pairwise_mds", "gpt_003_pairwise_mds"

## pairwise protest all elements - dsm

In [73]:
%%R 

matrices = list(leuven_dsm, w2v_dsm, clip_dsm, unverified_gpt_dsm, verified_gpt_dsm, human_triplet_dsm, gpt_triplet_dsm, gpt_embed_dsm, human_pairwise_dsm, gpt_003_pairwise_dsm, flan_xxl_triplet_dsm, flan_xxl_cot_triplet_dsm, flan_ul2_triplet_dsm, flan_ul2_cot_triplet_dsm, gpt_003_cot_triplet_dsm, gpt_002_cot_triplet_dsm, flan_ul2_pairwise_dsm, flan_ul2_pairwise_cot_dsm, gpt_003_pairwise_cot_dsm, gpt_002_pairwise_cot_dsm, gpt_002_pairwise_dsm, flan_xxl_dsm)

## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("leuven", "w2v", "clip", "unverified_gpt_summer_no_mc", "verified_gpt_summer_no_mc", "human_triplet", "gpt_triplet", "gpt_embed_mds",  "human_pairwise_mds", "gpt_003_pairwise_mds", "flan_xxl_triplet_mds", "flan_xxl_cot_triplet_mds", "flan_ul2_triplet_mds", "flan_ul2_cot_triplet_mds", "gpt_003_cot_triplet_mds", "gpt_002_cot_triplet_mds", "flan_ul2_pairwise_mds", "flan_ul2_cot_pairwise_mds", "gpt_003_pairwise_cot_mds", "gpt_002_pairwise_cot_mds", "gpt_002_pairwise_dsm", "flan_xxl_dsm")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        print(k)
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/dsm_the_whole_correlation_table.csv", sep=",")

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 

# final graphs for paper

### Tasks rsm for different model

#### humans

In [52]:
%%R 

matrices = list(leuven_dsm, human_triplet_dsm, human_pairwise_dsm)

## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("leuven",  "human_triplet", "human_pairwise_mds")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/humans_all_tasks_correlation_table.csv", sep=",")

#### text-davinci-002 no CoT

In [53]:
%%R 

matrices = list(verified_gpt_dsm, gpt_triplet_dsm, gpt_002_pairwise_dsm)

## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("verified_gpt_dsm",  "gpt_triplet_dsm", "gpt_002_pairwise_dsm")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/text-davinci-002_all_tasks_correlation_table.csv", sep=",")

#### flan-xxl no CoT

In [71]:
%%R 

matrices = list(flan_xxl_dsm, flan_xxl_triplet_dsm, flan_xxl_pairwise_dsm)

## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))){
        if (i==3 | j==3){
            results = c(results,0)
            }
        else{
            result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        }
        # Pass the two matrices to the protest function and store the result
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("flan_xxl_dsm",  "flan_xxl_triplet_dsm", "flan_xxl_pairwise_dsm")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/flan-xxl_all_tasks_correlation_table.csv", sep=",")

### Bar plot corr table

In [26]:
%%R 

matrices = list(leuven_dsm, w2v_dsm, clip_dsm, unverified_gpt_dsm, verified_gpt_dsm, human_triplet_dsm, gpt_triplet_dsm, gpt_embed_dsm, human_pairwise_dsm, gpt_003_pairwise_dsm, flan_xxl_triplet_dsm, flan_ul2_triplet_dsm, flan_ul2_pairwise_dsm , gpt_002_pairwise_dsm, flan_xxl_dsm)

## Create an empty list to store the results
results = list()

## Iterate through all combinations of the matrices
for (i in 1:(length(matrices))) {
    for (j in 1:(length(matrices))) {
        # Pass the two matrices to the protest function and store the result
        result = protest(matrices[[i]], matrices[[j]], symmetric=TRUE)
        results = c(results, sqrt(1-result$ss))
    }
}

## make the results into a table
## Create a data frame with all possible combinations of matrix names
matrix_names = list("leuven", "w2v", "clip", "unverified_gpt_summer_no_mc", "verified_gpt_summer_no_mc", "human_triplet", "gpt_triplet", "gpt_embed_mds",  "human_pairwise_mds", "gpt_003_pairwise_mds", "flan_xxl_triplet_mds",  "flan_ul2_triplet_mds", "flan_ul2_pairwise_mds", "gpt_002_pairwise_dsm", "flan_xxl_dsm")
matrix_combinations = expand.grid(matrix_names, matrix_names)

## Assign column names to the data frame
colnames(matrix_combinations) = c("matrix1", "matrix2")

## Fill in the values
k = 1
for (i in 1:length(matrix_names)) {
    for (j in 1:length(matrix_names)) {
        print(k)
        matrix_combinations[k, "result"] = results[k]
        k = k + 1
    }
}

protest_results =  pivot_wider(matrix_combinations, names_from = matrix1, values_from = result)
df <- apply(protest_results,2,as.character)

write.table(data.frame(df), "results/dsm_no_cot_the_whole_correlation_table.csv", sep=",")

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 